In [2]:
import numpy as np
import pandas as pd
import krippendorff
import json
import itertools

In [3]:
mojca= pd.read_csv(f'C:\\Users\\patri\\Desktop\\University\\AI\\combots\\Results-20221216T235732Z-001\\Results\\saved_df_mojca.csv', delimiter=';')

In [4]:
mojca

,Unnamed: 0.2,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,...,Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1.1,texts,coherence_results,consistency_results,relevance_results,fluency_results,comments
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,<br><br> Elena: Happy birthday my dear! <br><b...,5,5,4,5,NaN
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,<br><br> Elena: Happy birthday my dear! <br><b...,1,2,2,3,Just takes sentences from the original dialogu...
2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,<br><br> Elena: Happy birthday my dear! <br><b...,4,5,1,5,Does not capture some of the facts that are in...
3,3,3,3,3,3,3,3,3,3,3,...,3,3,3,3,<br><br> Elena: Happy birthday my dear! <br><b...,1,1,1,5,"Not true, doesnt capture any of the importantt..."
4,4,4,4,4,4,4,4,4,4,4,...,4,4,4,4,<br><br> Elena: Happy birthday my dear! <br><b...,3,1,1,3,"Not true fact, doesnt capture other important ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1395,1395,1395,1395,1395,1395,1395,1395,1395,1395,1395,...,1395,1395,1395,1395,<br><br> Karen: I passed!!!!! <br><br> Simon: ...,5,5,5,5,Any comments?
1396,1396,1396,1396,1396,1396,1396,1396,1396,1396,1396,...,1396,1396,1396,1396,<br><br> Karen: I passed!!!!! <br><br> Simon: ...,4,5,3,5,Any comments?
1397,1397,1397,1397,1397,1397,1397,1397,1397,1397,1397,...,1397,1397,1397,1397,<br><br> Karen: I passed!!!!! <br><br> Simon: ...,4,5,4,5,Any comments?
1398,1398,1398,1398,1398,1398,1398,1398,1398,1398,1398,...,1398,1398,1398,1398,<br><br> Karen: I passed!!!!! <br><br> Simon: ...,4,5,3,5,Any comments?


# load data

In [2]:
# our data
ann1 = pd.read_csv(f'data/saved_df_ann1.csv', delimiter=';')
ann2 = pd.read_csv(f'data/saved_df_ann2.csv', delimiter=';')
ann3 = pd.read_csv(f'data/saved_df_ann3.csv', delimiter=';')

# original data
fname = 'data/human_judgment.jsonl'
data = [] #NEWstart
with open(fname, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line.rstrip('\n|\r')))


In [15]:
# transform original data to match our data
original_annotator_scores= []

# a row in the data: id, dialogue, summary, annotations, model_id
# loop through annotator and each dimension 
for annotator in [0,1,2]:  
    scores = []  
    for type in ['consistency', 'coherence', 'fluency', 'relevance']:
        list_annotations=[]
        # store all annotations for an annotator 
        for row in data:
            annotations = row.get('annotations')
            list_annotations.append(annotations[annotator])
        dimension_list=[]
        # 
        for dimensions in list_annotations:
            dimension=dimensions.get(type.lower())
            dimension_list.append(dimension)
        scores.append(dimension_list)
    original_annotator_scores.append(scores)

orig_ann1 = pd.DataFrame(zip(original_annotator_scores[0][0], original_annotator_scores[0][1], original_annotator_scores[0][2], original_annotator_scores[0][3]), 
        columns=['consistency_results', 'coherence_results', 'fluency_results', 'relevance_results'])

orig_ann2 = pd.DataFrame(zip(original_annotator_scores[1][0], original_annotator_scores[1][1], original_annotator_scores[1][2], original_annotator_scores[1][3]), 
        columns=['consistency_results', 'coherence_results', 'fluency_results', 'relevance_results'])

orig_ann3 = pd.DataFrame(zip(original_annotator_scores[2][0], original_annotator_scores[2][1], original_annotator_scores[2][2], original_annotator_scores[2][3]), 
        columns=['consistency_results', 'coherence_results', 'fluency_results', 'relevance_results'])

In [16]:
list(orig_ann2['coherence_results']) == list(orig_ann3['coherence_results'])

False

# set up functions

In [17]:
# adjusted functions for our input data
def chonker(seq, size):
    return (seq[pos:pos+size] for pos in range(0, len(seq), size))

def filter_noise_scores(s_1, s_2, s_3):
    '''
        filter noise ratings of human and give the average value
        return numpy.array (100, 14)
    '''
    res = np.zeros((100, 14))
    for i, (r_1, r_2, r_3) in enumerate(zip(s_1, s_2, s_3)):
        for j, (a, b, c) in enumerate(zip(r_1, r_2, r_3)):
            if a == b and a != c:
                res[i,j] = a
            elif a == c and a != b:
                res[i,j] = c
            elif b == c and a != b:
                res[i,j] = b
            else:
                res[i,j] = (a + b + c) / 3
    return res

def no_noise_lists(s_1, s_2, s_3):
    '''
        filter noise ratings of human
    '''
    new_ann1 = []
    new_ann2 = []
    new_ann3 = []
    nans=0
    for (r_1, r_2, r_3) in zip(s_1, s_2, s_3):
        for (a, b, c) in zip(r_1, r_2, r_3):
            if a == b and a != c:
                new_ann1.append(a)
                new_ann2.append(b)
                new_ann3.append(np.nan)
                nans+=1
            elif a == c and a != b:
                new_ann1.append(a)
                new_ann2.append(np.nan)
                new_ann3.append(c)
                nans+=1
            elif b == c and a != b:
                new_ann1.append(np.nan)
                new_ann2.append(b)
                new_ann3.append(c)
                nans+=1
            else:
                new_ann1.append(a)
                new_ann2.append(b)
                new_ann3.append(c)

    print(4200-nans)
    return new_ann1, new_ann2, new_ann3

### Set up variables to do IAA and correlations 

In [18]:
# Our annotations
# transform the list of evaluation values into: [[n*14]*100]
coherence_ann1 = chonker(list(ann1['coherence_results']), 14)
coherence_ann2 = chonker(list(ann2['coherence_results']), 14)
coherence_ann3 = chonker(list(ann3['coherence_results']), 14)

consistency_ann1 = chonker(list(ann1['consistency_results']), 14)
consistency_ann2 = chonker(list(ann2['consistency_results']), 14)
consistency_ann3 = chonker(list(ann3['consistency_results']), 14)

fluency_ann1 = chonker(list(ann1['fluency_results']), 14)
fluency_ann2 = chonker(list(ann2['fluency_results']), 14)
fluency_ann3 = chonker(list(ann3['fluency_results']), 14)

relevance_ann1 = chonker(list(ann1['relevance_results']), 14)
relevance_ann2 = chonker(list(ann2['relevance_results']), 14)
relevance_ann3 = chonker(list(ann3['relevance_results']), 14)

# Original Annotators
# transform the list of evaluation values into: [[n*14]*100]
coherence_orig_ann1 = chonker(list(orig_ann1['coherence_results']), 14)
coherence_orig_ann2 = chonker(list(orig_ann2['coherence_results']), 14)
coherence_orig_ann3 = chonker(list(orig_ann3['coherence_results']), 14)

consistency_orig_ann1 = chonker(list(orig_ann1['consistency_results']), 14)
consistency_orig_ann2 = chonker(list(orig_ann2['consistency_results']), 14)
consistency_orig_ann3 = chonker(list(orig_ann3['consistency_results']), 14)

fluency_orig_ann1 = chonker(list(orig_ann1['fluency_results']), 14)
fluency_orig_ann2 = chonker(list(orig_ann2['fluency_results']), 14)
fluency_orig_ann3 = chonker(list(orig_ann3['fluency_results']), 14)

relevance_orig_ann1 = chonker(list(orig_ann1['relevance_results']), 14)
relevance_orig_ann2 = chonker(list(orig_ann2['relevance_results']), 14)
relevance_orig_ann3 = chonker(list(orig_ann3['relevance_results']), 14)

In [7]:
# noise filtering for IAA check
orig_coherence_nonoise1, orig_coherence_nonoise2, orig_coherence_nonoise3 = no_noise_lists(coherence_orig_ann1, coherence_orig_ann2, coherence_orig_ann3)
orig_consistency_nonoise1, orig_consistency_nonoise2, orig_consistency_nonoise3 = no_noise_lists(consistency_orig_ann1, consistency_orig_ann2, consistency_orig_ann3)
orig_fluency_nonoise1, orig_fluency_nonoise2, orig_fluency_nonoise3 = no_noise_lists(fluency_orig_ann1, fluency_orig_ann2, fluency_orig_ann3)
orig_relevance_nonoise1, orig_relevance_nonoise2, orig_relevance_nonoise3 = no_noise_lists(relevance_orig_ann1, relevance_orig_ann2, relevance_orig_ann3)


3161
3360
3050
3439


In [8]:
coherence_nonoise1, coherence_nonoise2, coherence_nonoise3 = no_noise_lists(coherence_ann1, coherence_ann2, coherence_ann3)
consistency_nonoise1,consistency_nonoise2,consistency_nonoise3 = no_noise_lists(consistency_ann1, consistency_ann2, consistency_ann3)
fluency_nonoise1, fluency_nonoise2,fluency_nonoise3 = no_noise_lists(fluency_ann1, fluency_ann2, fluency_ann3)
relevance_nonoise1,relevance_nonoise2,relevance_nonoise3 = no_noise_lists(relevance_ann1, relevance_ann2, relevance_ann3)

3607
3754
3625
3394


# calculations

### IAA calculations
using krippendorff's alpha

In [9]:
# Original IAA
coherence_IAA = krippendorff.alpha(reliability_data=[list(orig_ann1['coherence_results']),list(orig_ann2['coherence_results']),list(orig_ann3['coherence_results'])], level_of_measurement="interval")
fluency_IAA = krippendorff.alpha(reliability_data=[list(orig_ann1['fluency_results']),list(orig_ann2['fluency_results']),list(orig_ann3['fluency_results'])],level_of_measurement="interval")
consistency_IAA = krippendorff.alpha(reliability_data=[list(orig_ann1['consistency_results']),list(orig_ann2['consistency_results']),list(orig_ann3['consistency_results'])],level_of_measurement="interval")
relevance_IAA = krippendorff.alpha(reliability_data=[list(orig_ann1['relevance_results']),list(orig_ann2['relevance_results']),list(orig_ann3['relevance_results'])],level_of_measurement="interval")

print(f'Original IAA scores:\nCoherence: {coherence_IAA} \nConsistency: {consistency_IAA} \nFluency: {fluency_IAA} \nRelevance: {relevance_IAA}')

Original IAA scores:
Coherence: 0.3785576910327453 
Consistency: 0.492762621233651 
Fluency: 0.13361325787541334 
Relevance: 0.38671951755027045


In [10]:
coherence_IAA = krippendorff.alpha(reliability_data=[orig_coherence_nonoise1, orig_coherence_nonoise2, orig_coherence_nonoise3], level_of_measurement="interval")
fluency_IAA = krippendorff.alpha(reliability_data=[orig_fluency_nonoise1, orig_fluency_nonoise2, orig_fluency_nonoise3],level_of_measurement="interval")
consistency_IAA = krippendorff.alpha(reliability_data=[orig_consistency_nonoise1, orig_consistency_nonoise2, orig_consistency_nonoise3],level_of_measurement="interval")
relevance_IAA = krippendorff.alpha(reliability_data=[orig_relevance_nonoise1, orig_relevance_nonoise2, orig_relevance_nonoise3],level_of_measurement="interval")

print(f'Original IAA scores (cleaned):\nCoherence: {coherence_IAA} \nConsistency: {consistency_IAA} \nFluency: {fluency_IAA} \nRelevance: {relevance_IAA}')

Original IAA scores (cleaned):
Coherence: 0.7564020052749023 
Consistency: 0.6709247287723659 
Fluency: 0.6781873766563293 
Relevance: 0.5620838662718799


In [11]:
# Our IAA
coherence_IAA = krippendorff.alpha(reliability_data=[list(ann1['coherence_results']),list(ann2['coherence_results']),list(ann3['coherence_results'])], level_of_measurement="interval")
fluency_IAA = krippendorff.alpha(reliability_data=[list(ann1['fluency_results']),list(ann2['fluency_results']),list(ann3['fluency_results'])],level_of_measurement="interval")
consistency_IAA = krippendorff.alpha(reliability_data=[list(ann1['consistency_results']),list(ann2['consistency_results']),list(ann3['consistency_results'])],level_of_measurement="interval")
relevance_IAA = krippendorff.alpha(reliability_data=[list(ann1['relevance_results']),list(ann2['relevance_results']),list(ann3['relevance_results'])],level_of_measurement="interval")

print(f'Our IAA scores:\nCoherence: {coherence_IAA} \nConsistency: {consistency_IAA} \nFluency: {fluency_IAA} \nRelevance: {relevance_IAA}')

Our IAA scores:
Coherence: 0.6074598852525589 
Consistency: 0.7938139086383895 
Fluency: 0.5233497800176004 
Relevance: 0.5159398644189812


In [12]:
coherence_IAA = krippendorff.alpha(reliability_data=[coherence_nonoise1, coherence_nonoise2, coherence_nonoise3], level_of_measurement="interval")
fluency_IAA = krippendorff.alpha(reliability_data=[fluency_nonoise1, fluency_nonoise2, fluency_nonoise3],level_of_measurement="interval")
consistency_IAA = krippendorff.alpha(reliability_data=[consistency_nonoise1, consistency_nonoise2, consistency_nonoise3],level_of_measurement="interval")
relevance_IAA = krippendorff.alpha(reliability_data=[relevance_nonoise1, relevance_nonoise2, relevance_nonoise3],level_of_measurement="interval")

print(f'Our IAA scores (cleaned):\nCoherence: {coherence_IAA} \nConsistency: {consistency_IAA} \nFluency: {fluency_IAA} \nRelevance: {relevance_IAA}')

Our IAA scores (cleaned):
Coherence: 0.7780447258554797 
Consistency: 0.9227800328928509 
Fluency: 0.76903271937251 
Relevance: 0.7160557952595286


### Correlation between our and original results

In [20]:
# for each dimension filer out the noise and take the average
# our results
coherence_filtered = filter_noise_scores(coherence_ann1, coherence_ann2, coherence_ann3)
consistency_filtered = filter_noise_scores(consistency_ann1, consistency_ann2, consistency_ann3)
fluency_filtered = filter_noise_scores(fluency_ann1, fluency_ann2, fluency_ann3)
relevance_filtered = filter_noise_scores(relevance_ann1, relevance_ann2, relevance_ann3) 

# original results
orig_coherence_filtered = filter_noise_scores(coherence_orig_ann1, coherence_orig_ann2, coherence_orig_ann3)
orig_consistency_filtered = filter_noise_scores(consistency_orig_ann1, consistency_orig_ann2, consistency_orig_ann3)
orig_fluency_filtered = filter_noise_scores(fluency_orig_ann1, fluency_orig_ann2, fluency_orig_ann3)
orig_relevance_filtered = filter_noise_scores(relevance_orig_ann1, relevance_orig_ann2, relevance_orig_ann3)


In [21]:
# Original IAA
coherence_agreement = krippendorff.alpha(reliability_data=[list(itertools.chain(*coherence_filtered)), list(itertools.chain(*orig_coherence_filtered))], level_of_measurement="interval")
consistency_agreement = krippendorff.alpha(reliability_data=[list(itertools.chain(*consistency_filtered)), list(itertools.chain(*orig_consistency_filtered))], level_of_measurement="interval")
fluency_agreement = krippendorff.alpha(reliability_data=[list(itertools.chain(*fluency_filtered)), list(itertools.chain(*orig_fluency_filtered))], level_of_measurement="interval")
relevance_agreement = krippendorff.alpha(reliability_data=[list(itertools.chain(*relevance_filtered)), list(itertools.chain(*orig_relevance_filtered))], level_of_measurement="interval")

print(f'IAA scores between filtered results:\nCoherence: {coherence_agreement} \nConsistency: {fluency_agreement} \nFluency: {consistency_agreement} \nRelevance: {relevance_agreement}')

IAA scores between filtered results:
Coherence: 0.37876580164604223 
Consistency: 0.5209832222422306 
Fluency: 0.6828214886452186 
Relevance: 0.6492179949103607


In [22]:
list_coh_fil = list(itertools.chain(*coherence_filtered))
list_coh_fil_orig = list(itertools.chain(*orig_coherence_filtered))
print('Coherence correlation', np.corrcoef(list_coh_fil, list_coh_fil_orig))

Coherence correlation [[1.         0.41675892]
 [0.41675892 1.        ]]


In [23]:
list_con_fil = list(itertools.chain(*consistency_filtered))
list_con_fil_orig = list(itertools.chain(*orig_consistency_filtered))
print('Consistency correlation', np.corrcoef(list_con_fil, list_con_fil_orig))

Consistency correlation [[1.         0.76980152]
 [0.76980152 1.        ]]


In [24]:
list_flu_fil = list(itertools.chain(*fluency_filtered))
list_flu_fil_orig = list(itertools.chain(*orig_fluency_filtered))
print('Fluency correlation', np.corrcoef(list_flu_fil, list_flu_fil_orig))

Fluency correlation [[1.         0.54568637]
 [0.54568637 1.        ]]


In [25]:
list_rel_fil = list(itertools.chain(*relevance_filtered))
list_rel_fil_orig = list(itertools.chain(*orig_relevance_filtered))
print('Relevance correlation', np.corrcoef(list_rel_fil, list_rel_fil_orig))

Relevance correlation [[1.         0.68499235]
 [0.68499235 1.        ]]
